In [14]:
import pandas as pd
import numpy as np

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import warnings
import itertools

In [22]:
df = pd.read_excel('Data.xlsx', sheet_name='base')

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1433 entries, 0 to 1432
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Product ID                      1433 non-null   object        
 1   Week Date                       1433 non-null   datetime64[ns]
 2   Units Ordered                   1433 non-null   int64         
 3   Ordered Product Sales           1433 non-null   float64       
 4   Sessions                        1433 non-null   float64       
 5   BuyBox%                         1433 non-null   float64       
 6   Average_BuyBox%_Weekly          1433 non-null   float64       
 7   Sessions_with_Buy_Box           1433 non-null   float64       
 8   Sessions_without_Buy_Box        1433 non-null   float64       
 9   Units_per_Session_with_Buy_Box  1433 non-null   float64       
 10  Potential_Additional_Units      1430 non-null   float64       
 11  Pote

In [30]:
df

,Product ID,Week Date,Units Ordered,Ordered Product Sales,Sessions,BuyBox%,Average_BuyBox%_Weekly,Sessions_with_Buy_Box,Sessions_without_Buy_Box,Units_per_Session_with_Buy_Box,Potential_Additional_Units,Potential_Additional_Revenue,Lost_Revenue
0,P001,2023-12-31,2340,50041.8,13508.0,0.995114,0.985989,13442.0,66.0,0.0,0.0,0.0,0.0
1,P002,2023-12-31,1960,45250.0,16500.0,0.998667,0.985989,16478.0,22.0,0.0,0.0,0.0,0.0
2,P003,2023-12-31,1620,69700.0,8008.0,0.991758,0.985989,7942.0,66.0,0.0,0.0,0.0,0.0
3,P004,2023-12-31,620,19509.8,3058.0,0.964029,0.985989,2948.0,110.0,0.0,0.0,0.0,0.0
4,P005,2023-12-31,780,12207.0,6336.0,0.996528,0.985989,6314.0,22.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,P043,2024-09-22,40,5801.2,616.0,0.964286,0.905715,594.0,22.0,0.0,0.0,0.0,0.0
1429,P032,2024-09-22,20,2203.6,1166.0,0.905660,0.905715,1056.0,110.0,0.0,0.0,0.0,0.0
1430,P034,2024-09-22,20,1139.8,594.0,0.851852,0.905715,506.0,88.0,0.0,0.0,0.0,0.0
1431,P040,2024-09-22,20,1191.2,528.0,0.833333,0.905715,440.0,88.0,0.0,0.0,0.0,0.0


In [37]:
import pandas as pd
import numpy as np

def calculate_lost_revenue(df):
    """Calculates lost revenue, imputing missing BuyBox% and Sessions values."""

    df['Week Date'] = pd.to_datetime(df['Week Date'])

    # Handle missing Sessions using the weekly average
    df['Average_Sessions_Weekly'] = df.groupby('Week Date')['Sessions'].transform('mean')
    df['Sessions'] = df['Sessions'].fillna(df['Average_Sessions_Weekly'])

    # Handle missing BuyBox% using the weekly average
    df['Average_BuyBox%_Weekly'] = df.groupby('Week Date')['BuyBox%'].transform('mean')
    df['BuyBox%'] = df['BuyBox%'].fillna(df['Average_BuyBox%_Weekly'])

    df['Sessions_with_Buy_Box'] = df['Sessions'] * df['BuyBox%']
    df['Sessions_without_Buy_Box'] = df['Sessions'] - df['Sessions_with_Buy_Box']
    df['Units_per_Session_with_Buy_Box'] = df['Units Ordered'] / df['Sessions_with_Buy_Box']
    df['Potential_Additional_Units'] = df['Units_per_Session_with_Buy_Box'] * df['Sessions_without_Buy_Box']
    df['Potential_Additional_Revenue'] = df['Potential_Additional_Units'] * (df['Ordered Product Sales'] / df['Units Ordered'])
    df['Lost_Revenue'] = df['Potential_Additional_Revenue']

    df.drop(columns=['Average_Sessions_Weekly','Average_BuyBox%_Weekly','Potential_Additional_Revenue'], inplace=True)

    return df

In [38]:
df_output = calculate_lost_revenue(df)
print(f'Total Lost_Revenue: ${df_output["Lost_Revenue"].sum():,.2f}')
df_output.head()

Total Lost_Revenue: $1,401,445.97


,Product ID,Week Date,Units Ordered,Ordered Product Sales,Sessions,BuyBox%,Sessions_with_Buy_Box,Sessions_without_Buy_Box,Units_per_Session_with_Buy_Box,Potential_Additional_Units,Lost_Revenue
0,P001,2023-12-31,2340,50041.8,13508.0,0.995114,13442.0,66.0,0.174081,11.489362,245.704419
1,P002,2023-12-31,1960,45250.0,16500.0,0.998667,16478.0,22.0,0.118946,2.616822,60.413885
2,P003,2023-12-31,1620,69700.0,8008.0,0.991758,7942.0,66.0,0.203979,13.462604,579.224377
3,P004,2023-12-31,620,19509.8,3058.0,0.964029,2948.0,110.0,0.210312,23.134328,727.977612
4,P005,2023-12-31,780,12207.0,6336.0,0.996528,6314.0,22.0,0.123535,2.717770,42.533101


In [41]:
# Aggregate data by week
weekly_data = df.groupby('Week Date').agg(
    Sessions_without_Buy_Box=('Sessions_without_Buy_Box', 'sum'),
    Total_Lost_Revenue=('Lost_Revenue', 'sum')
).reset_index()

# Calculate weighted average Buy Box Missing %
#weekly_data['Weighted Avg Buy Box Missing %'] = weekly_data['Total_Revenue'] / weekly_data['Total_Potential_Revenue']

# Create the chart with two y-axes
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(
    x=weekly_data['Week Date'],
    y=weekly_data['Total_Lost_Revenue'],
    name="Total Lost Revenue",
    marker_color='blue'
))

fig.add_trace(go.Scatter(
    x=weekly_data['Week Date'],
    y=weekly_data['Sessions_without_Buy_Box'],  # Multiply by 100 to show as percentage
    name="Sessions without Buy Box",
    marker_color='red'
), secondary_y=True)

fig.update_layout(
    title="Weeklu Total Lost Revenue and Sessions without Buy Box",
    xaxis_title="Week Date",
    yaxis_title="Total Lost Revenue",
    yaxis2_title="Sessions without Buy Box",
    template="plotly_white"
)

fig.show()

In [8]:
def optimize_arima(sales, p_values, d_values, q_values, n_splits=3):
    """This function optimizes the parameters (p, d, q) for ARIMA using cross-validation."""
    best_rmse = float("inf")
    best_order = None
    ts_cv = TimeSeriesSplit(n_splits=n_splits)
    
    for order in itertools.product(p_values, d_values, q_values):
        try:
            rmse_scores = []
            for train_index, test_index in ts_cv.split(sales):
                train_sales, test_sales = sales.iloc[train_index], sales.iloc[test_index]
                with warnings.catch_warnings():
                    warnings.filterwarnings("ignore")
                    model = ARIMA(train_sales, order=order)
                    model_fit = model.fit()
                    predictions = model_fit.forecast(steps=len(test_sales))
                    rmse = np.sqrt(mean_squared_error(test_sales, predictions))
                    rmse_scores.append(rmse)
            avg_rmse = np.mean(rmse_scores)
            if avg_rmse < best_rmse:
                best_rmse = avg_rmse
                best_order = order
        except Exception as e:
            print(f"An error occurred with order {order}: {e}")
            continue
    return best_order

def forecast_sales(df, n_forecast=12, min_data_points=10):
    """This function forecasts sales using ARIMA with optimization for each product."""
    try:
        df['Week Date'] = pd.to_datetime(df['Week Date'])
        forecasts = []

        for product_id in df['Product ID'].unique():
            product_data = df[df['Product ID'] == product_id]
            product_data = product_data.groupby('Week Date')['Ordered Product Sales'].sum().reset_index()
            product_data = product_data.set_index('Week Date').sort_index()
            sales = product_data['Ordered Product Sales']

            if len(sales) < min_data_points:
                print(f"Not enough data for product {product_id}. Skipping.")
                continue

            all_weeks = pd.date_range(start=sales.index.min(), end=sales.index.max(), freq='W-SUN')
            sales = sales.reindex(all_weeks).fillna(0)

            p_values = range(0, 6)  
            d_values = range(0, 2)
            q_values = range(0, 3)  

            best_order = optimize_arima(sales, p_values, d_values, q_values)
            print(f"Mejor orden ARIMA para producto {product_id}: {best_order}")
            if best_order:
                try:
                    with warnings.catch_warnings():
                        warnings.filterwarnings("ignore")
                        model = ARIMA(sales, order=best_order)
                        model_fit = model.fit()
                        forecast = model_fit.forecast(steps=n_forecast)
                        last_date = sales.index[-1]
                        forecast_dates = pd.date_range(start=last_date + pd.Timedelta(days=7), periods=n_forecast, freq='W-SUN')
                        forecasts.append(pd.DataFrame({
                            'Product ID': product_id,
                            'Week Date': forecast_dates,
                            'Forecasted Sales': forecast
                        }))
                except Exception as e:
                    print(f"Error to adjust ARIMA for product {product_id}: {e}")
                    continue
            else:
                print(f"There is not a valid ARIMA for the product {product_id}.")
                continue

        if forecasts:
            forecasts_df = pd.concat(forecasts, ignore_index=True)
            return forecasts_df
        else:
            return None

    except Exception as e:
        print(f"An error occurred during forecasting: {e}")
        return None

In [9]:
def plot_forecasts(df, forecasts_df):
    """Plots actual and forecasted sales for each product with a dropdown.

    Args:
        df: Pandas DataFrame with original sales data.
        forecasts_df: Pandas DataFrame with forecasted sales.
    """
    if forecasts_df is None:
        print("No forecasts to plot.")
        return

    product_ids = df['Product ID'].unique()
    fig = go.Figure()

    for product_id in product_ids:
        product_data = df[df['Product ID'] == product_id].set_index('Week Date').sort_index()
        sales = product_data['Ordered Product Sales']

        forecasts_product = forecasts_df[forecasts_df['Product ID'] == product_id].set_index('Week Date').sort_index()
        if len(forecasts_product)>0:
            last_actual_date = sales.index.max()
            forecasts_product = forecasts_product[forecasts_product.index > last_actual_date]
            forecasts_sales = forecasts_product['Forecasted Sales']

            # Add traces (initially only show the first product)
            visibility = True if product_id == product_ids[0] else False
            fig.add_trace(go.Scatter(
                x=sales.index,
                y=sales,
                mode='lines',
                name=f"{product_id} - Actual Sales",
                line=dict(color='blue'),
                visible=visibility
            ))

            fig.add_trace(go.Scatter(
                x=forecasts_sales.index,
                y=forecasts_sales,
                mode='lines',
                name=f"{product_id} - Forecasted Sales",
                line=dict(color='red', dash='dash'),
                visible=visibility
            ))

    # Create dropdown buttons
    buttons = []
    for i, product_id in enumerate(product_ids):
        visibility = [False] * (2 * len(product_ids))  # Hide all traces
        visibility[2 * i] = True  # Show actual sales for this product
        visibility[2 * i + 1] = True  # Show forecasted sales for this product
        button = dict(
            label=product_id,
            method="update",
            args=[{"visible": visibility}],
        )
        buttons.append(button)

    # Add dropdown to layout
    fig.update_layout(
        updatemenus=[
            dict(
                active=0,
                buttons=buttons,
                direction="down",
                pad={"r": 10, "t": 10},
                showactive=True,
                x=0.1,
                xanchor="left",
                y=1.1,
                yanchor="top",
            )
        ],
        title="Sales Forecasts by Product",
        xaxis_title="Week Date",
        yaxis_title="Sales",
        template="plotly_white"
    )

    fig.show()

In [10]:
forecasts_df = forecast_sales(df)

Mejor orden ARIMA para producto P001: (3, 1, 1)
Mejor orden ARIMA para producto P002: (1, 0, 2)
Error con el orden (2, 0, 2): LU decomposition error.
Error con el orden (5, 0, 2): LU decomposition error.
Mejor orden ARIMA para producto P003: (4, 0, 2)
Error con el orden (5, 0, 2): LU decomposition error.
Error con el orden (5, 1, 2): LU decomposition error.
Mejor orden ARIMA para producto P004: (2, 1, 0)
Mejor orden ARIMA para producto P005: (1, 0, 1)
Error con el orden (1, 1, 2): LU decomposition error.
Error con el orden (2, 0, 0): LU decomposition error.
Error con el orden (2, 0, 1): LU decomposition error.
Error con el orden (2, 1, 1): LU decomposition error.
Error con el orden (3, 0, 1): LU decomposition error.
Error con el orden (3, 1, 1): LU decomposition error.
Error con el orden (4, 0, 2): LU decomposition error.
Error con el orden (4, 1, 0): LU decomposition error.
Error con el orden (5, 0, 2): LU decomposition error.
Error con el orden (5, 1, 0): LU decomposition error.
Mejo

In [11]:
plot_forecasts(df, forecasts_df)

In [ ]:
df.to_excel('q2.xlsx', sheet_name='question2', index=False)
forecasts_df.to_excel('q3.xlsx', sheet_name='question3', index=False)